# ❤️ Azure AI Agents

## Model Context Protocol (MCP) lab
![flow](../../utils/model-context-protocol.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/).

This lab includes the following MCP servers:
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.

<a id='testconnection'></a>
### 🧪 Test the connection to the MCP servers and List Tools

In [1]:
import os, json, asyncio, sys

sys.path.insert(1, "../../shared")  # add the shared directory to the Python path
import utils
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
from dotenv import load_dotenv
from datetime import datetime as pydatetime
from azure.ai.inference.models import UserMessage, SystemMessage
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

load_dotenv(override=True)

server_url = "https://" + f"{os.getenv('WEATHER_CONTAINERAPP_URL')}/weather/sse"

async def list_tools(server_url, authorization_header=None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    async with sse_client(server_url, headers) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            response = await session.list_tools()
            tools = response.tools
    return tools

tools = await list_tools(f"{server_url}")

print(f"✅ Connected to server {server_url}")
print("⚙️ Tools:")
for tool in tools:
    print(f"  - {tool.name}")
    print(f"     Input Schema: {tool.inputSchema}")

✅ Connected to server https://aca-weather-jrpn.yellowmoss-9594982e.eastus.azurecontainerapps.io/weather/sse
⚙️ Tools:
  - get_cities
     Input Schema: {'properties': {'country': {'title': 'Country', 'type': 'string'}}, 'required': ['country'], 'title': 'get_citiesArguments', 'type': 'object'}
  - get_weather
     Input Schema: {'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}


## 🧪 Run an OpenAI completion with MCP tools

In [2]:
# Azure AI Projects
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential, conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [3]:
nest_asyncio.apply()


async def call_tool(mcp_session, function_name, function_args):
    try:
        func_response = await mcp_session.call_tool(function_name, function_args)
        func_response_content = func_response.content
    except Exception as e:
        func_response_content = json.dumps({"error": str(e)})
    return str(func_response_content)


system_prompt = (
    "You are a helpful enterprise assistant at Contoso. "
    f"Today's date is {pydatetime.now().strftime('%A, %b %d, %Y, %I:%M %p')}. "
    "You have access to tools to get information about the weather, "
)


async def run_completion_with_tools(server_url, user_input):
    async with sse_client(server_url) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()
            response = await session.list_tools()
            tools = response.tools
            print(f"✅ Connected to server {server_url}")
            openai_tools = [
                {
                    "type": "function",
                    "function": {"name": tool.name, "parameters": tool.inputSchema},
                }
                for tool in tools
            ]

            # Step 1: send the conversation and available functions to the model
            print(
                "▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt"
            )
            messages = [{"role": "user", "content": user_input}]
            with project_client.inference.get_chat_completions_client() as chat_client:
                system_msg = SystemMessage(content=system_prompt)
                user_msg = UserMessage(content=user_input)

                response = chat_client.complete(
                    model="gpt-4o",
                    messages=[system_msg, user_msg],
                    tools=openai_tools,
                )

                response_message = response.choices[0].message
                tool_calls = response_message.tool_calls
                if tool_calls:
                    # Step 2: call the function
                    messages.append(
                        response_message
                    )  # extend conversation with assistant's reply
                    # send the info for each function call and function response to the model
                    print("▶️ Step 2: call the functions")
                    for tool_call in tool_calls:
                        function_name = tool_call.function.name
                        function_args = json.loads(tool_call.function.arguments)
                        print(
                            f"   Function Name: {function_name} Function Args: {function_args}"
                        )

                        function_response = await call_tool(
                            session, function_name, function_args
                        )
                        # Add the tool response
                        print(f"   Function response: {function_response}")
                        messages.append(
                            {
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": function_response,
                            }
                        )  # extend conversation with function response
                    print(
                        "▶️ Step 3: finish with a completion to anwser the user prompt using the function response"
                    )
                    # Step 3: finish with a completion to answer the user prompt using the function response
                    second_response = chat_client.complete(
                        model="gpt-4o",
                        messages=messages,
                    )  # get a new response from the model where it can see the function response
                    print("💬", second_response.choices[0].message.content)


asyncio.run(
    run_completion_with_tools(f"{server_url}", "What's the current weather in Medellin?")
)

✅ Connected to server https://aca-weather-jrpn.yellowmoss-9594982e.eastus.azurecontainerapps.io/weather/sse
▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt
▶️ Step 2: call the functions
   Function Name: get_weather Function Args: {'city': 'Medellin'}
   Function response: [TextContent(type='text', text="{'city': 'Medellin', 'condition': 'Windy', 'temperature': 28.23, 'humidity': 66.06}", annotations=None)]
▶️ Step 3: finish with a completion to anwser the user prompt using the function response
💬 The current weather in Medellin is windy with a temperature of approximately 28.23°C and a humidity level of 66.06%.
